In [1]:
import json
import numpy as np
import random
from math import trunc

# dictionary setup
sharpness_raw      = {'red':0.50, 'orange':0.75 , 'yellow':1.00, 'green':1.05, 'blue':1.20,   'white':1.32}
sharpness_element  = {'red':0.25, 'orange':0.50,  'yellow':0.75, 'green':1.00, 'blue':1.0625, 'white':1.15}
skills = {	'Critical Boost':{0:0.25, 1:0.30, 2:0.35, 3:0.40},
			'Weakness Exploit':{0:0, 1:0.15, 2:0.30, 3:0.50},
			'Critical Eye':{0:0, 1:0.05, 2:0.10, 3:0.15, 4:0.20, 5:0.25, 6:0.30, 7:0.40},
			'Attack Boost':{0:{'+':0, '*':0},    1:{'+':3, '*':0},    2:{'+':6, '*':0},    3:{'+':9,  '*':0},
							4:{'+':7, '*':0.05}, 5:{'+':8, '*':0.06}, 6:{'+':9, '*':0.08}, 7:{'+':10, '*':0.10}},
			'Agitator':{0:{'+':0,  '*':0},    1:{'+':4,  '*':0.03}, 2:{'+':8,  '*':0.05},
						3:{'+':12, '*':0.07}, 4:{'+':16, '*':0.10}, 5:{'+':20, '*':0.15}},
			'Heroics':{0:0, 1:0, 2:0.05, 3:0.05, 4:0.10, 5:0.30},
			'Dragonheart':{0:0, 1:0, 2:0,  3:0, 4:0.05, 5:0.10},
			'Resuscitate':{0:0, 1:5, 2:10, 3:20},
			'Resentment':{0:0, 1:5, 2:10, 3:15, 4:20, 5:25},
			'Peak Performance':{0:0, 1:5, 2:10, 3:20}}
			# critical draw
			# punishing draw
decorations = { 'Critical Boost':2, 'Weakness Exploit':2,
				'Critical Eye':2,   'Attack Boost':2,
				'Agitator':2,		'Heroics':2,
				'Resuscitate':2,	'Resentment':2,
				'Peak Performance':2}

with open('sets.json') as f:
	sets = json.load(f)
set_names = []
for key, value in sets.items():
	set_names.append(key)

def generate_first_generation(weapon):
	generation = dict()
	for i in range(0,30):
		generation[i] = generate_random_set(weapon)
	for i in range(len(generation)):
		try:
			generation['avg'] = generation['avg'] + generation[i]['efr']
		except KeyError:
			generation['avg'] = 0.0
	generation['avg'] = generation['avg'] / (len(generation) - 1)
	generation['improvement delta'] =  0
	return generation

def print_all(weapon, set, gen_count):
	print(f'Strongest efr of generation {gen_count}:')
	print(f"[{trunc(set['efr'])}]")
	print("weapon decorations:")
	print(weapon.decorations)
	print_armor_set(set)
	print_skills(set)
	print()

def print_skills(set):
	for skill in set['info']['skills']:
		print(skill, set['info']['skills'][skill])

# print individual armor set
def print_armor_set(armor):
	for piece in armor['set']:
		# name
		print(armor['set'][piece]['name'])
		# inherent skills
		for skill, level in armor['set'][piece]['skills'].items():
			print(f"\t-> {skill} {level}")
		# slot info
		print(f"\t-> {armor['set'][piece]['slots'][0]}-{armor['set'][piece]['slots'][1]}-{armor['set'][piece]['slots'][2]}")
		# slotted decorations
		try:
			for deco in armor['set'][piece]['decorations']:
				print(f"\t-> {deco}")
		except KeyError:
			None

# fitness function - calculating efr for a given set
def calc_efr(weapon, set_info):
	cb_lvl   = 0
	wex_lvl  = 0
	ce_lvl   = 0
	bonus_raw = 0
	for name, level in set_info['skills'].items():
		try:
			skills[name]
			if name == 'Critical Boost':
				if level > 3: cb_lvl = 3
				else: cb_lvl = level
			elif name == 'Weakness Exploit':
				if level > 3: wex_lvl = 3
				else: wex_lvl = level
			elif name == 'Critical Eye':
				if level > 7: ce_lvl = 7
				else: ce_lvl = level
			elif name == 'Attack Boost':
				if level > 7: bonus_raw += skills['Attack Boost'][7]['+']     + weapon.raw * skills['Attack Boost'][7]['*']
				else:		  bonus_raw += skills['Attack Boost'][level]['+'] + weapon.raw * skills['Attack Boost'][level]['*']
			elif name == 'Agitator': # divide by 2 as messy balancing
				if level > 5: bonus_raw += (skills['Agitator'][5]['+']     + weapon.raw * skills['Agitator'][5]['*'])/2
				else: 		  bonus_raw += (skills['Agitator'][level]['+'] + weapon.raw * skills['Agitator'][level]['*'])/2
			elif name == 'Heroics': # don't care about heroics right now
				if level > 5: bonus_raw += 0 #weapon.raw * skills['Heroics'][5]
				else:	 	  bonus_raw += 0 #weapon.raw * skills['Heroics'][level]
			elif name == 'Dragonheart':
				if level > 5: bonus_raw += weapon.raw * skills['Dragonheart'][5]
				else: 		  bonus_raw += weapon.raw * skills['Dragonheart'][level]
			elif name == 'Resuscitate':
				if level > 3: bonus_raw += skills['Resuscitate'][3]
				else:		  bonus_raw += skills['Resuscitate'][level]
			elif name ==  'Resentment': # divide by 2 as messy balancing
				if level > 5: bonus_raw += (skills['Resentment'][5])/2
				else:		  bonus_raw += (skills['Resentment'][level])/2
			elif name == 'Peak Performance':
				if level > 3: bonus_raw += skills['Peak Performance'][3]
				else:		  bonus_raw += skills['Peak Performance'][level]
		except KeyError: None
	critical_modifier = 1 + (weapon.affinity + skills['Weakness Exploit'][wex_lvl] + skills['Critical Eye'][ce_lvl]) * skills['Critical Boost'][cb_lvl]
	efr = (weapon.raw + bonus_raw) * sharpness_raw[weapon.sharpness] * critical_modifier
	return efr

# getting set information
def get_skills_decos_set(*args):
	info = {'skills':dict(), 'slots':{'lvl3':0, 'lvl2':0, 'lvl1':0}}
	for equipment in args:
		if type(equipment) == Weapon:
			for i in range(0,2):
				if weapon.slots[i] == 3:
					info['slots']['lvl3'] += 1
				if weapon.slots[i] == 2:
					info['slots']['lvl2'] += 1
				if weapon.slots[i] == 1:
					info['slots']['lvl1'] += 1
		else:
			if equipment['set']['helm']:
				get_skills_decos_piece(equipment['set']['helm'], info)
			if equipment['set']['chest']:
				get_skills_decos_piece(equipment['set']['chest'], info)
			if equipment['set']['arms']:
				get_skills_decos_piece(equipment['set']['arms'], info)
			if equipment['set']['waist']:
				get_skills_decos_piece(equipment['set']['waist'], info)
			if equipment['set']['legs']:
				get_skills_decos_piece(equipment['set']['legs'], info)
	return info

def get_skills_decos_piece(equipment, info):
	# get skills inherent to the equipment
	for skill, level in equipment['skills'].items():
		try:
			info['skills'][skill] += level
		except KeyError:
			info['skills'][skill] = level
	# get skills from decoration slots
	try:
		for skill in equipment['decorations']:
			try:
				info['skills'][skill] += 1
			except KeyError:
				info['skills'][skill] = 1
	except KeyError:
		None
	# get decoration info
	for i in range(0,2):
		if equipment['slots'][i] == 3:
			info['slots']['lvl3'] += 1
		elif equipment['slots'][i] == 2:
			info['slots']['lvl2'] += 1
		elif equipment['slots'][i] == 1:
			info['slots']['lvl1'] += 1

# generate one armor piece
def generate_random_piece(armor_type):
	while True:
		piece = sets[set_names[np.random.randint(0,145)]][armor_type]
		if piece != None: break
	piece['decorations'] = list()
	for slot in piece['slots']:
		decokeys = list(decorations.keys())
		random.shuffle(decokeys)
		for deco in decokeys:
			if decorations[deco] <= slot:
				piece['decorations'].append(deco)
				break
	return piece

def generate_decos(weapon):
	weapon.decorations = list()
	for slot in weapon.slots:
		decokeys = list(decorations.keys())
		random.shuffle(decokeys)
		for deco in decokeys:
			if decorations[deco] <= slot:
				weapon.decorations.append(deco)
				break

# generate one armor set
def generate_random_set(weapon):
	armor = {'set' :{'helm':dict(), 'chest':dict(),'arms':dict(),'waist':dict(),'legs':dict()},
			 'info':dict(),
			 'efr' :float()}
	generate_decos(weapon)
	armor['set']['helm']  = generate_random_piece('helm')
	armor['set']['chest'] = generate_random_piece('chest')
	armor['set']['arms']  = generate_random_piece('arms')
	armor['set']['waist'] = generate_random_piece('waist')
	armor['set']['legs']  = generate_random_piece('legs')
	armor['info'] = get_skills_decos_set(weapon, armor)
	armor['efr']  = calc_efr(weapon, armor['info'])
	return armor

# selection function - generate 20 random armor sets - sort them on EFR
def selection(weapon, generation):
	sort_generation(generation)
	next_gen = dict()
	alpha_ratio = generation[0]['efr'] / generation['avg']
	if alpha_ratio > 1.2: # 15 children, 10 from first, 5 from 3rd place. 15 random
		for i in range(0,10):
			next_gen[i] = crossover(generation[0], generation[1], weapon)
		for i in range(10,15):
			next_gen[i] = crossover(generation[0], generation[2], weapon)
		for i in range(15,30):
			next_gen[i] = generate_random_set(weapon)
	elif alpha_ratio > 1.15: # 10 children all from top 2. 20 random 
		for i in range(0,10):
			next_gen[i] = crossover(generation[0], generation[1], weapon)
		for i in range(10,30):
			next_gen[i] = generate_random_set(weapon)	
	elif alpha_ratio > 1.10: # 7 children from top 2. 23 random
		for i in range(0,7):
			next_gen[i] = crossover(generation[0], generation[1], weapon)
		for i in range(7,30):
			next_gen[i] = generate_random_set(weapon)
	elif alpha_ratio > 1.05: # 5 children from top 2. 25 random
		for i in range(0,5):
			next_gen[i] = crossover(generation[0], generation[1], weapon)
		for i in range(5,30):
			next_gen[i] = generate_random_set(weapon)
	else: # 3 children from top 2. 27 random
		for i in range(0,3):
			next_gen[i] = crossover(generation[0], generation[1], weapon)
		for i in range(3,30):
			next_gen[i] = generate_random_set(weapon)
	mutate(next_gen)
	# getting the average efr for the generation
	for i in range(len(next_gen)):
		try:
			next_gen['avg'] = next_gen['avg'] + next_gen[i]['efr']
		except KeyError:
			next_gen['avg'] = 0.0
	next_gen['avg'] = next_gen['avg'] / (len(next_gen) - 1)
	# retain the alpha set.
	if next_gen[0]['efr'] <= generation[0]['efr']:
		next_gen[0] = generation[0]
		try:
			next_gen['improvement delta'] = generation['improvement delta'] + 1
		except KeyError:
			next_gen['improvement delta'] = 1
	else:
		next_gen['improvement delta'] = 0
	return next_gen

# bubble sort
def sort_generation(generation):
	for set in generation:
		for j in range(len(generation) - 3):
			if generation[j]['efr'] < generation[j+1]['efr']:
				generation[j], generation[j+1] = generation[j+1], generation[j]

# crossover function, uniform crossover
def crossover(parent1, parent2, weapon):
	child = {'set' :{'helm':dict(), 'chest':dict(),'arms':dict(),'waist':dict(),'legs':dict()},
			 'info':dict(),
			 'efr' :float()}
	if random.random() >= 0.5:
			child['set']['helm'] = parent1['set']['helm']
	else:	child['set']['helm'] = parent2['set']['helm']
	if random.random() >= 0.5:
			child['set']['chest'] = parent1['set']['chest']
	else:	child['set']['chest'] = parent2['set']['chest']
	if random.random() >= 0.5:
			child['set']['arms'] = parent1['set']['arms']
	else:	child['set']['arms'] = parent2['set']['arms']
	if random.random() >= 0.5:
			child['set']['waist'] = parent1['set']['waist']
	else:	child['set']['waist'] = parent2['set']['waist']
	if random.random() >= 0.5:
			child['set']['legs'] = parent1['set']['legs']
	else:	child['set']['legs'] = parent2['set']['legs']
	child['info'] = get_skills_decos_set(weapon, child)
	child['efr']  = calc_efr(weapon, child['info'])
	return child	

# mutation function. one armor piece for each set in the generation will mutate
def mutate(generation):
	for i in range(1,len(generation)-2):
		rng = random.random()
		if rng >= 0 and rng < 0.2:
			generation[i]['set']['helmet']= generate_random_piece('helm')
		if rng >= 0.2 and rng < 0.4:
			generation[i]['set']['chest'] = generate_random_piece('chest')
		if rng >= 0.4 and rng < 0.6:
			generation[i]['set']['arms']  = generate_random_piece('arms')
		if rng >= 0.6 and rng < 0.8:
			generation[i]['set']['waist'] = generate_random_piece('waist')
		if rng >= 0.8 and rng <= 1:
			generation[i]['set']['legs']  = generate_random_piece('legs')

class Weapon:
	raw = 190
	affinity = 0.4
	sharpness = 'white'
	slots = [2,0,0]
	decorations = list()

class Talisman:
	slots = [2,2,0]

In [2]:
weapon   = Weapon()
talisman = Talisman()
gen_count = 0
#sss
# initial generation. totally random
generation = generate_first_generation(weapon)
#print(generation[0])
print_all(weapon, generation[0], gen_count)

Strongest efr of generation 0:
[288]
weapon decorations:
['Critical Boost']
Channeler's Hair-tie
	-> Razor Sharp 1
	-> Spare Shot 1
	-> 0-0-0
Izuchi Mail
	-> Critical Eye 1
	-> Recovery Speed 1
	-> 0-0-0
Azure Age Sleeve Alpha  
	-> Constitution 1
	-> Critical Element 2
	-> Weakness Exploit 1
	-> 0-0-0
Death Stench Bowels
	-> Dragon Resistance 1
	-> 0-0-0
Bishaten Greaves
	-> Maximum Might 1
	-> 0-0-0
Razor Sharp 1
Spare Shot 1
Critical Eye 1
Recovery Speed 1
Constitution 1
Critical Element 2
Weakness Exploit 1
Dragon Resistance 1
Maximum Might 1



In [3]:
# evolve until it no improvement is seen for <epoch> generations
epoch = 1000
while generation['improvement delta'] < epoch:
	generation = selection(weapon, generation)
	gen_count += 1
print_all(weapon, generation[0], gen_count)

Strongest efr of generation 2250:
[413]
weapon decorations:
['Critical Eye']
Kaiser Crown
	-> Critical Eye 3
	-> Critical Boost 1
	-> 1-0-0
Valstrax Mail
	-> Dragonheart 1
	-> Resuscitate 2
	-> 3-1-0
	-> Resentment
Valstrax Braces
	-> Dragonheart 1
	-> Weakness Exploit 2
	-> 2-0-0
	-> Resuscitate
Volvidon Coil
	-> Rapid Morph 1
	-> 0-0-0
Hunter's Greaves S
	-> Stun Resistance 1
	-> Attack Boost 2
	-> 2-0-0
	-> Critical Eye
Critical Eye 3
Critical Boost 2
Dragonheart 2
Resuscitate 3
Weakness Exploit 3
Resentment 1
Attack Boost 4
Stun Resistance 1

